In [1]:
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from astropy.io import fits
from astropy.table import Table
import optuna
import pickle
import matplotlib.pyplot as plt


/home/21953404Victor/SDC3/tutorial-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Cargar los datos a partir de los archivos FITS
path_datos = "/home/21953404Victor/SDC3/SDC3GIT/data_actualizada/"
path_repo="/home/21953404Victor/SDC3/SDC3GIT/"
path = path_datos + "ZW3.msw_image.fits"
hdul = fits.open(path)

Transformación del catálogo de fuentes completo (almacenado)

In [3]:

# ruta_carpeta_catalogos = "/home/21953404Victor/SDC3/SDC3GIT/catalogos/catalogo completo 2/resultados/resultados_detection4"
# ruta_carpeta_resultados = "/home/21953404Victor/SDC3/SDC3GIT/catalogos/catalogo completo 2/labels"
# archivos = os.listdir(ruta_carpeta_catalogos)

# # Función para crear un archivo de anotación para cada catálogo
# def crear_anotaciones(path_cat, path_anotacion):
#     cat = Table.read(path_cat, format="ascii")
#     clase = "fuente"

#     with open(path_anotacion, "w") as f:
#         for row in cat:
#             x = row["X_IMAGE"]
#             y = row["Y_IMAGE"]
#             f.write(f"{x} {y} {clase}\n")

# # Procesar todos los archivos de catálogo en la carpeta
# for archivo in archivos:
#     # Extraer el número de kHz del nombre del archivo
#     num_khz = re.search(r"(\d+)kHz", archivo).group(1)

#     # Crear un nombre para el archivo de anotación correspondiente
#     anotacion_nombre = f"catalogo_completo_1_{num_khz}kHz.txt"

#     # Crear el archivo de anotación para el catálogo actual
#     crear_anotaciones(
#         os.path.join(ruta_carpeta_catalogos, archivo),
#         os.path.join(ruta_carpeta_resultados, anotacion_nombre)
#    )


In [4]:
# ruta_carpeta_resultados = "/home/21953404Victor/SDC3/SDC3GIT/catalogos/catalogo completo 1/labels"
# archivos = os.listdir(ruta_carpeta_resultados)

# def clamp(value, min_value, max_value):
#     return max(min(value, max_value), min_value)

# def procesar_archivo(path_txt):
#     with open(path_txt, "r") as f:
#         lines = f.readlines()

#     label = np.zeros((2048, 2048), dtype=int)

#     for line in lines:
#         x, y, _ = line.split()
#         x, y = int(float(x)), int(float(y))

#         for i in range(-4, 4):
#             for j in range(-4, 4):
#                 x_coord = clamp(x + i, 0, 2047)
#                 y_coord = clamp(y + j, 0, 2047)
#                 label[y_coord, x_coord] = 1

#     return label

# # Crear una lista para guardar los datos procesados
# target=[]

# # Procesar todos los archivos de texto en la carpeta de resultados
# for archivo in archivos:
#     path_txt = os.path.join(ruta_carpeta_resultados, archivo)
#     label = procesar_archivo(path_txt)
#     target.append(label)


In [7]:
# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
torch.cuda.empty_cache()

## Transformar los datos

In [7]:
# Cargar los datos y dividirlos en conjuntos de entrenamiento y validación
num_samples = len(hdul[0].data) # Cargar todos los datos en lugar de una muestra de todas las imágenes
all_data = hdul[0].data
indices = sorted(np.random.choice(all_data.shape[0], num_samples, replace=False))
data = all_data[indices]

input_data = np.array(data).astype("float32") / 255.0 # Normalizar los datos de entrada (en formato de 8 bits implica que cad píxel tenga un valor de 0 a 255)
input_data = input_data.reshape((data.shape[0], 1, data.shape[1], data.shape[2]))

del all_data
del hdul
del data

# En lugar de repetir el label ahora se carga la lista de labels. El resto del proceso es igual
output_data = np.array(fits.open(path_repo+'catalogos/catalogo completo 2/fit/LABEL_COMPLETO_V2.fit')[0].data).astype("float32")#[:, np.newaxis, :, :] # Agregar una dimensión para mantener el formato de los datos de entrada de la red neuronal

x_train, x_val, y_train, y_val = train_test_split(input_data, output_data, test_size=0.2, random_state=42)

del input_data
del output_data

x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train).unsqueeze(1)
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val).unsqueeze(1)

In [5]:
def create_decoder(dropout_rate, out_channels1, out_channels2, out_channels3):
    decoder = nn.Sequential(
        nn.ConvTranspose2d(out_channels3, out_channels2, 3, stride=2, padding=1, output_padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(out_channels2),
        nn.Dropout(dropout_rate),
        nn.ConvTranspose2d(out_channels2, out_channels1, 3, stride=2, padding=1, output_padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(out_channels1),
        nn.Dropout(dropout_rate),
        nn.ConvTranspose2d(out_channels1, 1, 3, padding=1),
        nn.Sigmoid()
    )
    return decoder.to(device)

def create_encoder(input_channels, num_filters1, num_filters2, dropout_rate, out_channels3):
    encoder = nn.Sequential(
        nn.Conv2d(input_channels, num_filters1, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(num_filters1),
        nn.MaxPool2d(2, 2),
        nn.Dropout(dropout_rate),
        nn.Conv2d(num_filters1, num_filters2, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(num_filters2),
        nn.MaxPool2d(2, 2),
        nn.Dropout(dropout_rate),
        nn.Conv2d(num_filters2, out_channels3, 3, padding=1),
        nn.ReLU(),
        nn.Dropout(dropout_rate)
    )
    return encoder.to(device)

### Optimizador de hiperparámetros

In [9]:
def objective(trial):
    # Cargar y dividir tus datos en conjuntos de entrenamiento y validación
    train_data = TensorDataset(x_train, y_train)
    val_data = TensorDataset(x_val, y_val)    
    
    # Crear DataLoaders para entrenamiento y validación
    tam_lote = 1
    train_loader = DataLoader(train_data, batch_size=tam_lote, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=tam_lote, shuffle=False)

    # Hiperparámetros sugeridos por Optuna
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.5)
    
    # Aquí también sugerimos el número de canales en las capas de convolución del decoder
    out_channels1 = trial.suggest_int("out_channels1", 16, 64)
    out_channels2 = trial.suggest_int("out_channels2", 16, 64)
    out_channels3 = trial.suggest_int("out_channels3", 16, 64)
    
    # Sugerir el número de filtros en las capas de convolución del encoder
    num_filters1 = trial.suggest_int("num_filters1", 16, 64)

    num_filters2 = trial.suggest_int("num_filters2", 16, 64)

    # Imprimir los hiperparámetros utilizados en este trial
    print(f"Hiperparámetros del trial {trial.number}:")
    print(f"lr: {lr}, dropout_rate: {dropout_rate}")
    print(f"out_channels1: {out_channels1}, out_channels2: {out_channels2}, out_channels3: {out_channels3}")
    print(f"num_filters1: {num_filters1}, num_filters2: {num_filters2}")

    # Crear el modelo de 'encoder' y 'decoder' con los hiperparámetros sugeridos
    input_channels = 1
    encoder = create_encoder(input_channels, num_filters1, num_filters2, dropout_rate, out_channels3)
    decoder = create_decoder(dropout_rate, out_channels1, out_channels2, out_channels3)

    # Función de pérdida y optimizador
    criterion = nn.BCELoss()
    optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=lr)

    # Crear un directorio para guardar los modelos intermedios, si aún no existe
    os.makedirs("modelos_intermedios", exist_ok=True)

    # Entrenamiento y validación del modelo
    num_epochs = 50
    train_losses = []
    val_losses = []

    try:
        for epoch in range(num_epochs):
            encoder.train()
            decoder.train()
            train_loss = 0.0

            #train
            
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()

                enc_output = encoder(images)
                dec_output = decoder(enc_output)

                # Convertir las etiquetas a float
                labels_float = labels.float()

                loss = criterion(dec_output, labels_float)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                
            train_losses.append(train_loss / len(train_loader))
            
            encoder.eval()
            decoder.eval()
            val_loss = 0.0

            #test
            
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    
                    enc_output = encoder(images)
                    dec_output = decoder(enc_output)
                    dec_output = torch.squeeze(dec_output, dim=1) 
                    
                    # Convertir las etiquetas a float
                    labels_float = labels.float()
                    labels_float = torch.squeeze(labels_float, dim=1)
                    
                    loss = criterion(dec_output, labels_float)
                    
                    val_loss += loss.item()
            
            val_losses.append(val_loss / len(val_loader))
            
            print(f"Epoch {epoch+1}/{num_epochs}, Pérdida del train: {train_losses[-1]}, Pérdida del test: {val_losses[-1]}")

        # Guardar los modelos intermedios
        trial_number = trial.number
        decoder_path = f"modelos_intermedios/decoder_gpu_cat_completo_trial_{trial_number}.pt"
        encoder_path = f"modelos_intermedios/encoder_gpu_cat_completo_trial_{trial_number}.pt"
        
        torch.save(decoder.state_dict(), decoder_path)
        torch.save(encoder.state_dict(), encoder_path)

        # Limpiar la GPU
        del decoder
        del encoder
        torch.cuda.empty_cache()
        
        # Guardar el plot de pérdidas por epoch
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.savefig(f"modelos_intermedios/loss_plot_trial_{trial_number}.png")
        plt.close()

        return val_losses[-1]  # Devolver la pérdida de validación del último epoch
    
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(f"[W] El trial {trial.number} falló debido a un error de falta de memoria en la GPU")
            print("Continuando con la optimización de parámetros, ignorando esta última configuración")
            
            # Limpiar la GPU
            del decoder
            del encoder
            torch.cuda.empty_cache()
            
            # Asignar un valor alto para que este trial no sea seleccionado como el mejor
            return float("inf")
        else:
            # Vuelve a generar la excepción si el error no es "CUDA out of memory"
            raise e

In [10]:
# Crear un estudio de optimización
study = optuna.create_study(direction="minimize")

# Ejecutar el estudio de optimización con la función 'objective' y un número determinado de ensayos
study.optimize(objective, n_trials=10)


[I 2023-05-22 21:56:23,818] A new study created in memory with name: no-name-da4f3137-0232-4807-8efb-f323913dea1b
/tmp/ipykernel_59891/1417001533.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
/tmp/ipykernel_59891/1417001533.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.0, 0.5)


Hiperparámetros del trial 0:
lr: 0.0009758457565748282, dropout_rate: 0.34667579337970894
out_channels1: 43, out_channels2: 26, out_channels3: 51
num_filters1: 40, num_filters2: 56
Epoch 1/50, Pérdida del train: 0.10779799414902097, Pérdida del test: 97.72823557142395
Epoch 2/50, Pérdida del train: 0.04167606332080646, Pérdida del test: 97.42884472588808
Epoch 3/50, Pérdida del train: 0.039118013049786286, Pérdida del test: 1.6033684351167625
Epoch 4/50, Pérdida del train: 0.0375366442060719, Pérdida del test: 0.7136143660018457


In [16]:
with open('study.pkl', 'wb') as f:
    pickle.dump(study, f)

# post entrenamiento

In [40]:
with open('study.pkl', 'rb') as f:
    study = pickle.load(f)

print("Mejor prueba: ")
print(study.best_trial)

print("\nMejores hiperparámetros: ")
print(study.best_trial.params)

Mejor prueba: 
FrozenTrial(number=2, state=TrialState.COMPLETE, values=[0.16273348724973793], datetime_start=datetime.datetime(2023, 5, 23, 0, 42, 24, 873186), datetime_complete=datetime.datetime(2023, 5, 23, 1, 50, 31, 341843), params={'lr': 0.00010027415861131253, 'dropout_rate': 0.09585204102585937, 'out_channels1': 44, 'out_channels2': 28, 'out_channels3': 26, 'num_filters1': 61, 'num_filters2': 17}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=True, low=1e-05, step=None), 'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.0, step=None), 'out_channels1': IntDistribution(high=64, log=False, low=16, step=1), 'out_channels2': IntDistribution(high=64, log=False, low=16, step=1), 'out_channels3': IntDistribution(high=64, log=False, low=16, step=1), 'num_filters1': IntDistribution(high=64, log=False, low=16, step=1), 'num_filters2': IntDistribution(high=64, log=False, low=16, step=1)}, trial_id=2, value=None)

Me

In [41]:
# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print("Tasa de aprendizaje:", study.best_params["lr"])
print("Tasa de dropout:", study.best_params["dropout_rate"])
print("Número de filtros en la primera capa convolucional del encoder:", study.best_params["num_filters1"])
print("Número de filtros en la segunda capa convolucional del encoder:", study.best_params["num_filters2"])
print("Número de canales en la primera capa del decoder:", study.best_params["out_channels1"])
print("Número de canales en la segunda capa del decoder:", study.best_params["out_channels2"])
print("Número de canales en la tercera capa del decoder:", study.best_params["out_channels3"])

# Obtener el número del mejor ensayo
best_trial_number = study.best_trial.number

# Crear un nuevo 'decoder' y 'encoder' con los mejores hiperparámetros encontrados
decoder = create_decoder(
    study.best_params["dropout_rate"],
    study.best_params["out_channels1"],
    study.best_params["out_channels2"],
    study.best_params["out_channels3"])

encoder = create_encoder(
    1, # número de canales de entrada
    study.best_params["num_filters1"],
    study.best_params["num_filters2"],
    study.best_params["dropout_rate"],
    study.best_params["out_channels3"]
    )

# Cargar los estados de los mejores modelos
decoder_path = f"modelos_intermedios/decoder_gpu_cat_completo_trial_{best_trial_number}.pt"
encoder_path = f"modelos_intermedios/encoder_gpu_cat_completo_trial_{best_trial_number}.pt"

decoder.load_state_dict(torch.load(decoder_path))
encoder.load_state_dict(torch.load(encoder_path))

Mejores hiperparámetros encontrados:
Tasa de aprendizaje: 0.00010027415861131253
Tasa de dropout: 0.09585204102585937
Número de filtros en la primera capa convolucional del encoder: 61
Número de filtros en la segunda capa convolucional del encoder: 17
Número de canales en la primera capa del decoder: 44
Número de canales en la segunda capa del decoder: 28
Número de canales en la tercera capa del decoder: 26


<All keys matched successfully>

In [47]:
import pandas as pd

df = pd.DataFrame()
for i, trial in enumerate(study.trials, 1):
    row = trial.params
    row['loss last 20'] = round(trial.value, 4)
    row['rank'] = i
    row['nº ensayo'] = str(trial.number)
    if 'channels' in row:
        row['ch.'] = row.pop('channels')
    if 'filters' in row:
        row['filt.'] = row.pop('filters')
    if 'lr' in row:
        row['learning rate'] = round(row.pop('lr'), 4)
    df = df.append(row, ignore_index=True)

for col in df.columns:
    if df[col].dtype.kind in 'fc':
        df[col] = df[col].round(4)

df = df.sort_values(by='loss last 20')
df['rank'] = range(1, len(df) + 1)
df.set_index('rank', inplace=True)


/tmp/ipykernel_74009/1435658589.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_74009/1435658589.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_74009/1435658589.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_74009/1435658589.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/tmp/ipykernel_74009/1435658589.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use

In [51]:
# Guarda el DataFrame como un archivo Excel
df.to_excel('ranking_modelos.xlsx', index=False, engine="openpyxl")


ModuleNotFoundError: No module named 'openpyxl'